**19BCE2433 PRITI SRI S**

In [4]:
import sklearn
import numpy as np
import pandas as pd

In [5]:
from google.colab import files
train_data=files.upload()

Saving train.csv to train (1).csv


In [7]:
test_data=files.upload()

Saving test.csv to test.csv


In [8]:
import io
train = pd.read_csv(io.BytesIO(train_data['train.csv']))
test=pd.read_csv(io.BytesIO(test_data['test.csv']))

In [9]:
train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [10]:
sum(train["label"] == 0)
sum(train["label"] == 1)

10413

In [11]:
train.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [12]:
train=train.dropna()
train=train.copy()
train.reset_index(inplace=True)


In [6]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [22]:
ps = PorterStemmer()
def news(df):
  corpus = []
  for i in range(0, len(train)):
    review = re.sub('[^a-zA-Z]', ' ', train['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)
  return corpus

In [26]:
nltk.download('stopwords')
train_fake = news(train["text"])
train_fake = pd.DataFrame(train_fake)
train["news"] = train_fake

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
test_fake = news(test["text"])
test_fake = pd.DataFrame(test_fake)
test["news"] = test_fake
test.tail()

,id,title,author,text,news
5195,25995,The Bangladeshi Traffic Jam That Never Ends - ...,Jody Rosen,Of all the dysfunctions that plague the world’...,year old donald trump brick everi lie hillari ...
5196,25996,John Kasich Signs One Abortion Bill in Ohio bu...,Sheryl Gay Stolberg,WASHINGTON — Gov. John Kasich of Ohio on Tu...,democrat weak bench undermin hope take back se...
5197,25997,"California Today: What, Exactly, Is in Your Su...",Mike McPhate,Good morning. (Want to get California Today by...,make davo class trump era new york time
5198,25998,300 US Marines To Be Deployed To Russian Borde...,NaN,« Previous - Next » 300 US Marines To Be Deplo...,espn b debacl footbal semifin rate breitbart
5199,25999,"Awkward Sex, Onscreen and Off - The New York T...",Teddy Wayne,Perhaps you’ve seen the new TV series whose pi...,anti hillari ad power su get taken watch late


In [28]:
from sklearn.model_selection import train_test_split
y = train.label.values
x_train, x_test, y_train, y_test = train_test_split(train.news.values, y,stratify=y, random_state=1, test_size=0.3, shuffle=True)
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(binary=True, stop_words='english')
vectorizer.fit(list(x_train) + list(x_test))
x_train_vec = vectorizer.transform(x_train)
x_test_vec = vectorizer.transform(x_test)

In [29]:
from sklearn import svm
svm = svm.SVC(kernel = 'linear', probability=True)
prob = svm.fit(x_train_vec, y_train).predict_proba(x_test_vec)
y_pred_svm = svm.predict(x_test_vec)

In [30]:
from sklearn.metrics import accuracy_score
print("Accuracy score for SVM is: ", accuracy_score(y_test, y_pred_svm) * 100, '%')

Accuracy score for SVM is:  92.70871308786 %


In [34]:
from sklearn.metrics import confusion_matrix, precision_score , recall_score , f1_score
ps1=precision_score(y_test, y_pred_svm)
print("Precision Score of the given dataset is :",(ps1*100),'%')


Precision Score of the given dataset is : 90.36341363821968 %


In [39]:
cm = confusion_matrix(y_test, y_pred_svm)
print('The confusion matrix is:')
print(cm)

The confusion matrix is:
[[2873  236]
 [ 164 2213]]


In [41]:
rs1=recall_score(y_test, y_pred_svm)
print("The recall score is:",rs1*100,'%')

The recall score is: 93.10054690786706 %
